*Giménez Arias, Carolina*

*Pons Suñer, Pedro*

## Consumer-Producer

Este script debería utilizarse tras ejecutar el script *TweetProducer*.

En este script se leen los tweets guardados en el topic *tweetsRaw*, se procesan y son guardados en un segundo topic, *tweetsProcessed*, para luego ser consumidos en los scripts *TweetsConsumer-Barplot* y *TweetsConsumer-TimeSeries*.

In [ ]:
# Cargamos las librerías a usar:
import matplotlib.pyplot as plt
from confluent_kafka import Consumer, KafkaError, Producer
from IPython.display import display, clear_output
import pandas as pd
import time
import json

In [ ]:
# Creamos un productor
p = Producer({
    'bootstrap.servers': 'localhost:9092',
})

In [ ]:
#Función que guardará los datos en el topic tweetsProcessed
def put_to_stream(data):
    p.poll(0)
    payload = json.dumps(data).encode('utf-8') #Importante para evitar problemas en la lectura: codificación utf-8
    p.produce('tweetsProcessed', payload)

In [ ]:
#Definimos el consumidor:
cons = Consumer({
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'mygroupCP'
})

#Nos suscribimos al topic en el que el primer productor inserta tweets:
cons.subscribe(['tweetsRaw'])

while True:
    
    msg = cons.poll(1.0) #espera un segundo, si en ese segundo nos llega algo lo usa
    if msg is None:
        continue
    
    tweet_info=msg.value().decode('utf-8') #Muy importante: codificación utf-8
    tweet_info=tweet_info.replace('@Pablo_Iglesias_','PODEMOS')
    tweet_info=tweet_info.replace('@sanchezcastejon','PSOE')
    tweet_info=tweet_info.replace('Pedro Sanchez','PSOE')
    tweet_info=tweet_info.replace('@Albert_Rivera','CIUDADANOS')
    tweet_info=tweet_info.replace('@pablocasado_','PP')
    tweet_info=tweet_info.replace('@populares','PP')
    tweet_info=tweet_info.replace('@CiudadanosCs','CIUDADANOS')
    tweet_info=tweet_info.replace('@ahorapodemos','PODEMOS')
    tweet_info=tweet_info.replace('@Santi_ABASCAL','VOX')
    tweet_info=tweet_info.replace('Vox','VOX')
    tweet_info=tweet_info.replace('vox','VOX')
    tweet_info=tweet_info.replace('Psoe','PSOE')
    tweet_info=tweet_info.replace('psoe','PSOE')
    tweet_info=tweet_info.replace('Cs','CIUDADANOS')
    tweet_info=tweet_info.replace('Ciudadanos','CIUDADANOS')
    tweet_info=tweet_info.replace('Podemos','PODEMOS')
    tweet_info=tweet_info.replace('pp','PP')
    #tweet_info=tweet_info.upper() #Pasamos todo a mayúscula para facilitar la búsqueda de los términos que aparecen tanto en mayus como minus
    put_to_stream(tweet_info)
    
cons.close()